In [ ]:
#create folder structure

In [ ]:
%cd /Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/TensorFlow

In [9]:
!git config --global http.postBuffer 524288000
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 90351, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 90351 (delta 72), reused 128 (delta 64), pack-reused 90205
Receiving objects: 100% (90351/90351), 608.42 MiB | 4.32 MiB/s, done.
Resolving deltas: 100% (65087/65087), done.


In [ ]:
!export PYTHONPATH=$PYTHONPATH:/path/to/models

## Preparing the Dataset 

In [10]:
%pip install labelImg

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for labelImg: filename=labelImg-1.8.6-py2.py3-none-any.whl size=261540 sha256=67074b35a7c8a5117f11ea144730c578312f5789f3c6f3fce71f67c4400ccd55
  Stored in directory: /Users/trinhtruc/Library/Caches/pip/wheels/a0/2d/29/aaf47d232f5b03ec3e64a8432032f328dfdea1714041712bf6
Successfully built labelImg

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/b

In [2]:
%cd dataset


/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/dataset


In [ ]:
# Rename image file code
import os

def rename_files(directory):
    # List all files in the directory
    files = os.listdir(directory)
    t = 1
    for file in files:
        ext_file = file.split(".")[-1]
        if (ext_file != "DS_Store"):
            # Construct the new file name by adding a prefix
            new_name = f"{str(t)}.{ext_file}"

            # # Build the full paths
            old_path = os.path.join(directory, file)
            new_path = os.path.join(directory, new_name)

            # # Rename the file
            os.rename(old_path, new_path)
            print(f'Renamed: {old_path} to {new_path}')
            t = t + 1

# Specify the directory and prefix
directory_path = './'

# Rename files in the specified directory with the given prefix
rename_files(directory_path)

In [9]:
!labelImg

Image:/Users/trinhtruc/Downloads/VOCdevkit/CatDog/000001.jpg -> Annotation:/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/dataset/000001.xml
Image:/Users/trinhtruc/Downloads/VOCdevkit/CatDog/000011.jpg -> Annotation:/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/dataset/000011.xml
Image:/Users/trinhtruc/Downloads/VOCdevkit/CatDog/000018.jpg -> Annotation:/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/dataset/000018.xml
Image:/Users/trinhtruc/Downloads/VOCdevkit/CatDog/000108.jpg -> Annotation:/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample/dataset/000108.xml
Image:/Users/trinhtruc/Downloads/VOCdevkit/CatDog/000157.jpg -> Annotation:/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectD

## Partition the Dataset

In [11]:
%cd ../

/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample


In [15]:
import os
import re
from shutil import copyfile
import argparse
import math
import random


def iterate_dir(source, dest, ratio, copy_xml):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)

    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        filename = images[idx]
        copyfile(os.path.join(source, filename),
                 os.path.join(test_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

    for filename in images:
        copyfile(os.path.join(source, filename),
                 os.path.join(train_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(train_dir, xml_filename))

In [16]:
iterate_dir("./dataset", "./TensorFlow/workspace/training_demo/images", 0.1, True)

/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_44639/4076859984.py:21: DeprecationWarning: Flags not at the start of the expression '([a-zA-Z0-9\\s_\\\\.\\-\\' (truncated)
  if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]


## Create Label Map

In [17]:
!pwd

/Users/trinhtruc/Documents/STUDY/Khoa học dữ liệu ảnh_Nền tảng và Ứng dụng/230104026_Lab2/ObjectDetectionExample


In [18]:
arr_labels = ['cat', 'dog']

with open('TensorFlow/workspace/training_demo/annotations/label_map.pbtxt', 'w') as f:
    for i, label in enumerate(arr_labels, 1):
        f.write(f"item {{\n")
        f.write(f"    id: {i}\n")
        f.write(f"    name: '{label}'\n")
        f.write(f"}}\n\n")

print("label_map.pbtxt file created successfully.")

label_map.pbtxt file created successfully.


## Create TensorFlow Records

### Convert *.xml to *.record

In [ ]:
%pip install object_detection

In [ ]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str)
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default=None)
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)


def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)
        for member in root.findall('object'):
            bndbox = member.find('bndbox')
            value = (filename,
                     width,
                     height,
                     member.find('name').text,
                     int(bndbox.find('xmin').text),
                     int(bndbox.find('ymin').text),
                     int(bndbox.find('xmax').text),
                     int(bndbox.find('ymax').text),
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):

    writer = tf.python_io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = xml_to_csv(args.xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))
    if args.csv_path is not None:
        examples.to_csv(args.csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(args.csv_path))


if __name__ == '__main__':
    tf.app.run()

In [ ]:
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str)
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default=None)
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

In [ ]:
images_xmls_train_path = "./Tensorflow/workspace/training_demo/images/train"
images_xmls_test_path = "./Tensorflow/workspace/training_demo/images/test"
output_record_anotation_train_path = "./Tensorflow/workspace/training_demo/annotations/train.record"
output_record_anotation_test_path = "./Tensorflow/workspace/training_demo/annotations/test.record"

writer = tf.python_io.TFRecordWriter(args.output_path)
path = os.path.join(args.image_dir)
examples = xml_to_csv(args.xml_dir)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
writer.close()
print('Successfully created the TFRecord file: {}'.format(args.output_path))
if args.csv_path is not None:
    examples.to_csv(args.csv_path, index=None)
    print('Successfully created the CSV file: {}'.format(args.csv_path))

In [ ]:
# python generate_tfrecord.py -x [PATH_TO_IMAGES_FOLDER]/train -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map.pbtxt -o [PATH_TO_ANNOTATIONS_FOLDER]/train.record


In [ ]:
# python generate_tfrecord.py -x "./Tensorflow/workspace/training_demo/images/train" -l "./Tensorflow/workspace/training_demo/annotations/label_map.pbtxt" -o "./Documents/Tensorflow/workspace/training_demo/annotations/train.record"
# python generate_tfrecord.py -x "./Tensorflow/workspace/training_demo/images/test" -l "./Tensorflow/workspace/training_demo/annotations/label_map.pbtxt" -o "./Documents/Tensorflow/workspace/training_demo/annotations/test.record"

